In [1]:
import pandas as pd
path = "data/gp_practice_weighted_population.xlsx"  # excel file containing the gp practice level data
data = pd.read_excel(path, sheet_name ='GP practice WP by ICS', header = 0, usecols="F,L,M:AC")  # Dataframe with specific columns that will be used

In [2]:
# Manipulate loaded dataframe

data = data.rename(columns={"STP21_42": "ICS", "GP practice name": "practice_name"})  # Rename some columns with more sensible names
data["Practice"] = data["Practice"] + " " + ":" + " " + data["practice_name"]  # Concatenate practice name with practice code to ensure all practices are unique

In [6]:
# Session state initialisation and variables, this ensures that everytime a user adds a place, the previous places remain.
col_list = list(data.columns.to_list())  # create a list of columns exactly the same as those in the original data for the output df

In [7]:
col_list = col_list.append("Place_Name")  # add a Place Name column which will be used to group practices by defined place
output_df = pd.DataFrame(columns=col_list)  # initialise empty output dataframe with defined column names
output_df
lists=[]
places=[]  # initialise session state, empty df that will hold places and empty list that will hold assigned practices
flat_list = [item for sublist in lists for item in sublist]  # session state list is a list of lists so this unpacks them into one single flat list to use later


In [10]:
# Drop downs for user manipulation/selection of data
ics = data['ICS'].drop_duplicates()  # pandas series of unique ICSs for dropdown list
ics = ics.sort_values()  # sort ICSs in alphabetical order
ics_choice = "Cumbria and North East"  # dropdown for selecting ICS
practices = list(data["Practice"].loc[data["ICS"] == ics_choice])  # dynamic list of practices that changes based on selected ICS
practices = [x for x in practices if x not in flat_list]  # this removes practices that have been assigned to a place from the practices dropdown list
practice_choice = ["A83005 : Whinfield Medical Practice", "A83006 : Orchard Court Surgery"]
place_name = "Group 1"

In [16]:

place_practices = list(practice_choice)  # Assign the practices in the newly defined place to a list
df_1 = data.query("Practice == @place_practices")  # Queries the original data and only returns the selected practices
df_1["Place Name"] = place_name  # adds the place name to the dataframe to allow it to be used for aggregation
df_1


<ipython-input-16-849b57c797d7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1["Place Name"] = place_name  # adds the place name to the dataframe to allow it to be used for aggregation


,ICS,Practice,practice_name,GP_pop,WP_G&A,WP_CS,WP_MH,WP_Mat,WP_HCHS,MFF_index,EACA_index,WP_MFF,WP_EACA,WP_Presc,WP_AM,Target_exc_remote_£k,Target_inc_remote_£k,WP_Overall,Need_index,Place Name
0,Cumbria and North East,A83005 : Whinfield Medical Practice,Whinfield Medical Practice,11832.000000,14223.289062,14635.094727,13431.027344,10383.470703,13971.007288,0.944639,0.997954,11176.968969,11807.794679,13719.485352,14287.391264,16745.507963,16745.507963,13335.857051,1.127101,Group 1
1,Cumbria and North East,A83006 : Orchard Court Surgery,Orchard Court Surgery,8035.083496,8809.822266,8601.802734,8451.464844,8028.321777,8710.220464,0.944639,0.997954,7590.253457,8018.645719,8750.815430,9782.611422,10571.726654,10571.726654,8419.155498,1.047799,Group 1


In [18]:
df_2 = df_1.groupby('Place Name').agg(
    {'GP_pop': 'sum', 'WP_G&A': 'sum', 'WP_CS': 'sum', 'WP_MH': 'sum', 'WP_Mat': 'sum', 'WP_HCHS': 'sum', 'EACA_index' : 'sum', "WP_Presc": 'sum', "WP_AM": 'sum', "WP_Overall": "sum"})  # aggregates the practices to give the aggregated place values
df_2 = df_2.apply(round)
df_2

,GP_pop,WP_G&A,WP_CS,WP_MH,WP_Mat,WP_HCHS,EACA_index,WP_Presc,WP_AM,WP_Overall
Place Name,,,,,,,,,,
Group 1,19867.0,23033.0,23237.0,21882.0,18412.0,22681.0,2.0,22470.0,24070.0,21755.0
